In [26]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv("/Users/anchanghun/Downloads/AI_Study/dataset/Applestock/AAPL.csv")


In [39]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.100178,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094952,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087983,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.090160,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.092774,73449600


In [40]:
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
10463,2022-06-13,132.869995,135.199997,131.440002,131.880005,131.880005,122207100
10464,2022-06-14,133.130005,133.889999,131.479996,132.759995,132.759995,84784300
10465,2022-06-15,134.289993,137.339996,132.160004,135.429993,135.429993,91533000
10466,2022-06-16,132.080002,132.389999,129.039993,130.059998,130.059998,108123900
10467,2022-06-17,130.070007,133.080002,129.809998,131.559998,131.559998,134118500


In [42]:
df.drop("Date", axis=1)

,Open,High,Low,Close,Adj Close,Volume
0,0.128348,0.128906,0.128348,0.128348,0.100178,469033600
1,0.122210,0.122210,0.121652,0.121652,0.094952,175884800
2,0.113281,0.113281,0.112723,0.112723,0.087983,105728000
3,0.115513,0.116071,0.115513,0.115513,0.090160,86441600
4,0.118862,0.119420,0.118862,0.118862,0.092774,73449600
...,...,...,...,...,...,...
10463,132.869995,135.199997,131.440002,131.880005,131.880005,122207100
10464,133.130005,133.889999,131.479996,132.759995,132.759995,84784300
10465,134.289993,137.339996,132.160004,135.429993,135.429993,91533000
10466,132.080002,132.389999,129.039993,130.059998,130.059998,108123900


In [48]:
open_axis = df["Open"]
high_axis = df["High"]
low_axis = df["Low"]
close_axis = df["Close"]
adj_axis = df["Adj Close"]
vol_axis = df["Volume"]

open_axis = open_axis.values
high_axis = high_axis.values
low_axis = low_axis.values
close_axis = close_axis.values
adj_axis = adj_axis.values
vol_axis = vol_axis.values

In [49]:
vol_axis.shape

(10468,)

In [50]:
# multivariate output data prep
from numpy import array
from numpy import hstack

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [52]:
# convert to [rows, columns] structure
open_axis = open_axis.reshape((len(open_axis), 1))
high_axis = high_axis.reshape((len(high_axis), 1))
low_axis = low_axis.reshape((len(low_axis), 1))
adj_axis = adj_axis.reshape((len(adj_axis), 1))
vol_axis = vol_axis.reshape((len(vol_axis), 1))
out_seq = close_axis.reshape((len(close_axis), 1))



# horizontally stack columns
dataset = hstack((open_axis, high_axis, low_axis,vol_axis,adj_axis,out_seq))

# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)


(10465, 3, 6) (10465, 6)


In [54]:
# define model

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.layers import concatenate

n_features = X.shape[2]

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(n_features))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(X, y, epochs=3000, verbose=0)


In [56]:
# open, high, low, close(종가), vol, adj(output, 종가)
x_input = array([[192.30, 195.33, 192.26, 193.89, 60130000, 193.89],
                [195.02, 195.33, 193.83, 195.18, 42200000, 195.18],
                 [195.42, 196.38, 194.34, 194.5, 53460000, 194.5]
                ])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[-2.8895979e+00  1.3253592e+02 -5.3611465e+01  3.6461272e+03
   5.1262820e+07 -3.1038336e+04]]
